In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import csv
import pandas as pd
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# read the csv data file
df = pd.read_csv('/content/drive/My Drive/You_trash/bgg-13m-reviews.csv',index_col=0)
# select particular columns
bgg_r = df[["user","rating","ID",'name']]
#Add year to our ds
ds_score = pd.read_csv('/content/drive/My Drive/You_trash/2019-05-02.csv',index_col=0)
score = ds_score[['Name', "Year","URL"]]
#We remove old versions of games
url = score.groupby("Name")['URL'].apply(list).reset_index(name='URL')
score = score.groupby("Name")['Year'].apply(list).reset_index(name='Year')
score = score.merge(url,left_on='Name',right_on="Name")
score["Year"] = score["Year"].apply(max)
score = score.rename(columns={"Name": "name"})
bgg_r = bgg_r.merge(score,left_on='name',right_on="name")
#We add average rating
g = bgg_r.groupby('ID', as_index=False)['rating'].mean()
g = g.rename(columns={"rating": "Bayes average"})
bgg_r = bgg_r.merge(g,left_on='ID',right_on="ID")
# rename game ID column
bgg_r = bgg_r.rename(columns={'ID': 'gameId'})

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
bgg_r

,user,rating,gameId,name,Year,URL,Bayes average
0,sidehacker,10.0,13,Catan,1995,[/boardgame/13/catan],7.211526
1,Varthlokkur,10.0,13,Catan,1995,[/boardgame/13/catan],7.211526
2,dougthonus,10.0,13,Catan,1995,[/boardgame/13/catan],7.211526
3,cypar7,10.0,13,Catan,1995,[/boardgame/13/catan],7.211526
4,ssmooth,10.0,13,Catan,1995,[/boardgame/13/catan],7.211526
...,...,...,...,...,...,...,...
13170068,pswissler,3.0,8256,LEGO Constructionary Game,1999,[/boardgame/8256/lego-constructionary-game],4.566667
13170069,Sprayoncrayon,3.0,8256,LEGO Constructionary Game,1999,[/boardgame/8256/lego-constructionary-game],4.566667
13170070,PhoenixSong,2.5,8256,LEGO Constructionary Game,1999,[/boardgame/8256/lego-constructionary-game],4.566667
13170071,tsantos,2.0,8256,LEGO Constructionary Game,1999,[/boardgame/8256/lego-constructionary-game],4.566667


##Work with smaller DataSet

In [ ]:
#extra filtered
bgg_r2 = bgg_r[bgg_r.groupby("gameId")['gameId'].transform('size') > 200]
#removing old games (usualy they not available for purchase)
bgg_r2 = bgg_r2[bgg_r["Year"]>1995]
#We setting minimal threshold for user
bgg_r2 = bgg_r2[bgg_r.groupby("user")['user'].transform('size') > 15]
# bgg_r2 = bgg_r2[bgg_r.groupby("user")['user'].transform('size') < 500]
# bgg_r2 = bgg_r2[bgg_r.groupby("user")['Bayes average'].transform('size') > 5]

# drop rows with null values
bgg_r2 = bgg_r2.dropna()

# create new ids for users and games (this is used for building the sparse matrix)
user_c2 = pd.Categorical(bgg_r2.user)
game_c2 = pd.Categorical(bgg_r2.gameId)

# add new id rows
bgg_r2['userIdx'] = user_c2.codes
bgg_r2['gameIdx'] = game_c2.codes

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [ ]:
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype
n_games2 = game_c2.categories.size
n_users2 = user_c2.categories.size
# build user-item sparse matrix
ui_sparse_matrix2 = csr_matrix((bgg_r2["rating"], (bgg_r2['userIdx'], bgg_r2['gameIdx'])), shape=(n_users2, n_games2))

# build dataframe from user-item sparse matrix
df_ui_matrix2 = pd.DataFrame.sparse.from_spmatrix(ui_sparse_matrix2)
df_ui_matrix2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,5227,5228,5229,5230,5231,5232,5233,5234,5235,5236,5237,5238,5239,5240,5241,5242,5243,5244,5245,5246,5247,5248,5249,5250,5251,5252,5253,5254,5255,5256,5257,5258,5259,5260,5261,5262,5263,5264,5265,5266
0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.5,0.0,0.0,6.5,0.0,0.0,8.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0
1,4.0,0.0,4.0,6.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,4.0,5.0,6.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8.0,0.0,0.0,7.5,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.5,0.0,0.0,7.0,0.0,7.0,7.5,0.0,8.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.5,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
134746,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
134747,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
134748,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_games = bgg_r2[['gameId','name']].drop_duplicates()
df_ratings = bgg_r2[['userIdx','gameId','Bayes average']].drop_duplicates()
R = df_ui_matrix2.values
user_ratings_mean = np.mean(R, axis = 1)
R = R - user_ratings_mean.reshape(-1, 1)
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R, k = 50)
sigma = np.diag(sigma)
all_user_predicted_ratings = user_ratings_mean.reshape(-1, 1) + np.dot(np.dot(U, sigma), Vt)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = df_ui_matrix2.columns)

In [ ]:
U.shape

(134750, 50)

In [ ]:
sigma.shape

(50, 50)

In [ ]:
def recommend_games(preds_df, userId, df_games = df_games, original_ratings_df = df_ratings, num_recommendations=15):
    user_row_number = userId - 1 
    sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False)
    user_data = original_ratings_df[original_ratings_df.userIdx == (userId)]
    user_full = (user_data.merge(df_games, how = 'left', left_on = 'gameId', right_on = 'gameId').
                     sort_values(['Bayes average'], ascending=False)
                 )
    recommendations = (df_games[~df_games['gameId'].isin(user_full['gameId'])]).merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left', left_on = 'gameId',
               right_on = 'index').rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations]
                      

    return user_full, recommendations

In [ ]:
def recommend(ID):
  Your_id = bgg_r2[bgg_r2['user'] =='%s'%ID]["userIdx"].iloc[0]
  already_rated, predictions = recommend_games(preds_df, Your_id,df_games = df_games, original_ratings_df = df_ratings, num_recommendations = 10)
  url =[]
  for line in predictions['name']:
   url.append( 'https://boardgamegeek.com/' + bgg_r2[bgg_r2['name'] == line]["URL"].iloc[0][0])
  predictions['URL'] = url
  return already_rated, predictions,url

# already_rated, predictions,url = recommend ('edroz')
# print(already_rated.head(10))
# print (predictions.head(20))

In [ ]:
already_rated, predictions,url = recommend ('edroz')
# print(already_rated.head(10))
predictions.head(10)

In [ ]:
already_rated, predictions,url = recommend ('DemYank')
# print(already_rated.head(10))
predictions.head(10)

In [ ]:
already_rated, predictions,url = recommend ('AquaSulis')
# print(already_rated.head(10))
predictions.head(10)

,gameId,name,index,Predictions,URL
3267,912,Pacific Victory,912.0,4.399678,https://boardgamegeek.com//boardgame/912/pacif...
1175,357,Excape,357.0,1.695378,https://boardgamegeek.com//boardgame/357/excape
2123,3487,BattleTech CCG,3487.0,1.457245,https://boardgamegeek.com//boardgame/3487/batt...
2739,1412,Trendy,1412.0,1.180448,https://boardgamegeek.com//boardgame/1412/trendy
4404,3627,Chaostle,3627.0,0.954221,https://boardgamegeek.com//boardgame/3627/chao...
3872,4106,Moderne Zeiten,4106.0,0.905269,https://boardgamegeek.com//boardgame/4106/mode...
2470,3269,Clue: The Card Game,3269.0,0.613932,https://boardgamegeek.com//boardgame/3269/clue...
19,15,Cosmic Encounter,15.0,0.512527,https://boardgamegeek.com//boardgame/39463/cos...
2018,2542,Gargon,2542.0,0.502731,https://boardgamegeek.com//boardgame/2542/gargon
3388,3749,Sumo!,3749.0,0.415735,https://boardgamegeek.com//boardgame/3749/sumo


In [ ]:
already_rated, predictions,url = recommend ('jbaileymusic')
# print(already_rated.head(10))
predictions.head(10)